In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import glob

In [3]:
import matplotlib.pyplot as plt
#%matplotlib notebook

In [4]:
from IPython.html import widgets
from IPython.html.widgets import interact
from IPython.display import display

/Users/j35/anaconda/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
from PyQt4 import QtGui

In [6]:
import pyfits
import shutil

## Select the working folder

In [7]:
%gui qt

In [8]:
try:
    from PySide import QtCore, QtGui
except ImportError:
    from PyQt4 import QtCore, QtGui

def gui_fname(dir=None, message=None):
    """Select a directory
    """
    if dir is None: dir ='../'
    if message is None: message = 'Select Folder ...'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, message,
                                                      dir,
                                                      QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [9]:
%gui qt
data_folder = gui_fname()
%matplotlib inline

In [ ]:
# remove hidden file .DS_Store 
ds_file = os.path.join(data_folder, ".DS_Store")
try:
    os.remove(ds_file)
except:
    pass

In [ ]:
list_of_files = [name for name in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, name))]
number_of_files = len(list_of_files)
print('Folder contains %d files.' %number_of_files)

### check that files have been loaded in the right order

In [ ]:
from pprint import pprint
pprint(list_of_files)

#### !!! wrong! files are in the wrong ordre !!!!

## Sort files (accroding to the last digits of their name)

#### files should be sorted as following
1. Lid_spr_1.fits
2. Lid_spr_10.fits
3. Lid_spr_11.fits
4. Lid_spr_20.fits
5. Lid_spr_100.fits

to do so, we need to write a method that will correctly format the last digits of the name as following

1. Lid_spr_0001.fits
2. Lid_spr_0010.fits
3. Lid_spr_0011.fits
4. Lid_spr_0100.fits

In [ ]:
def format_filename(filename):
    base, ext = os.path.splitext(filename)
    filename_split = base.split('_')
    index = filename_split[-1]
    index_formatted = "%.4d" % int(index)
    filename_split[-1] = index_formatted
    new_filename = "_".join(filename_split)
    return new_filename + ext

## try
#filename = list_of_files[0]
#print(format_filename(filename))

In [ ]:
list_of_files_reformatted = []
for file in list_of_files:
    new_file_name = format_filename(file)
    list_of_files_reformatted.append(new_file_name)

In [ ]:
pprint(list_of_files_reformatted)

## Renaming the files

#### Select where to mv the new renamed files

In [10]:
%gui qt
renamed_folder = gui_fname(dir=data_folder)
%matplotlib inline

In [ ]:
pbar = widgets.IntProgress()
pbar.max = len(list_of_files)
display(pbar)

for index, file in enumerate(list_of_files):
    old_full_file_name = data_folder + '/' + file
    new_file = list_of_files_reformatted[index]
    new_full_file_name = renamed_folder + '/' + new_file
#    print("%s -> %s" %(old_full_file_name, new_full_file_name))
    shutil.copyfile(old_full_file_name, new_full_file_name)
    pbar.value = index


## Loading the data in the right order this time

In [11]:
list_of_files_sorted = glob.glob(renamed_folder + '/*.fits')
## pprint(list_of_files_sorted)

In [12]:
pbar = widgets.IntProgress()
pbar.max = len(list_of_files_sorted)
display(pbar)

list_images = []
for index, file in enumerate(list_of_files_sorted):
    o_fits = pyfits.open(file)
    o_fits_0 = o_fits[0]
    list_images.append(o_fits_0.data)
    pbar.value = index
    
#print(len(list_images))